In [9]:
!python -m venv myenv

!source myenv/bin/activate

In [10]:
!pip install seal
!pip install python-dp

DEPRECATION: Loading egg at /home/jun/.pyenv/versions/3.11.8/lib/python3.11/site-packages/seal-4.0.0-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /home/jun/.pyenv/versions/3.11.8/lib/python3.11/site-packages/seal-4.0.0-py3.11-linux-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330


In [11]:
import sys
sys.path.append('/home/jun/.pyenv/versions/3.11.8/lib/python3.11/site-packages')

In [20]:
from seal import EncryptionParameters, SEALContext, Encryptor, Plaintext, Ciphertext, BatchEncoder

def encrypt_data(data, context, public_key):
    encoder = BatchEncoder(context)
    encryptor = Encryptor(context, public_key)
    plaintext = Plaintext()
    encoder.encode(data, plaintext)
    encrypted_data = Ciphertext()
    encryptor.encrypt(plaintext, encrypted_data)
    return encrypted_data


In [23]:
from seal import Evaluator

def compute_on_encrypted_data(encrypted_data1, encrypted_data2, context):
    evaluator = Evaluator(context)
    encrypted_result = Ciphertext()
    evaluator.add(encrypted_data1, encrypted_data2, encrypted_result)
    return encrypted_result


In [24]:
from seal import Decryptor, Plaintext, BatchEncoder
from pydp.algorithms.laplacian import BoundedMean

def decrypt_and_apply_dp(encrypted_data, context, secret_key, epsilon):
    decryptor = Decryptor(context, secret_key)
    plain_result = Plaintext()
    decryptor.decrypt(encrypted_data, plain_result)
    
    # BatchEncoderを使用してデコード
    encoder = BatchEncoder(context)
    decoded_result = encoder.decode_int32(plain_result)
    
    # PyDPを用いて差分プライバシーのノイズを加える
    # 最初の要素のみを使用する場合（もし複数のデータポイントがない場合）
    dp_result = BoundedMean(epsilon, 0, 100)
    dp_result.add_entry(decoded_result[0])
    return dp_result.result()
